In [ ]:
!pip install -q --upgrade "transformers>=4.30.0" accelerate bitsandbytes safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.9 MB/s eta 0:00:00


In [ ]:
!pip install -q --upgrade huggingface_hub pyngrok fastapi "uvicorn[standard]" nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.2/452.2 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 7.5 MB/s eta 0:00:00


In [ ]:
from getpass import getpass
import os

os.environ['HF_TOKEN'] = getpass()

··········


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,
    low_cpu_mem_usage=True
)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear8bitLt(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear8bitLt(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear8bitLt(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear8bitLt(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-0

In [ ]:
import torch

prompt = "What is the capital of Japan?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=250)
text = tokenizer.decode(out[0], skip_special_tokens=True)
text

'What is the capital of Japan? The capital of Japan is Tokyo. It is the largest city in Japan and serves as the country\'s political, economic, and cultural center. Tokyo is located in the eastern part of the country and is known for its modern infrastructure, vibrant urban life, and historical significance. \n\nNote: While Tokyo is the capital, some may refer to "the capital of Japan" in a broader sense, such as the historical capital of Kyoto, which has been the imperial capital for centuries. However, officially, Tokyo is the capital of Japan. \n\n✅ Final Answer: **Tokyo**.'

In [ ]:
import secrets

print(secrets.token_urlsafe(32))

In [ ]:
from fastapi import FastAPI, Request, Header, HTTPException

app = FastAPI()
SECRET = ""

@app.post("/generate")
async def generate(request: Request, authorization: str = Header(None)):
    if authorization != f"Bearer {SECRET}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    data = await request.json()
    prompt = data.get("input", "")
    if not prompt:
        return {"error": "no input provided"}

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=500)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return {"text": text}


In [ ]:
os.environ['NGROK_AUTHTOKEN'] = getpass()

··········


In [ ]:
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import threading

nest_asyncio.apply()
port = 8002
public_url = ngrok.connect(port).public_url
print("Public URL:", public_url)

def run():
    uvicorn.run(app, host="0.0.0.0", port=port)

threading.Thread(target=run, daemon=True).start()

Public URL: https://coud-chemosynthetic-floy.ngrok-free.dev
